In [1]:
%load_ext autoreload
%autoreload 2

In [50]:
import urban_dictionary_scraper
import torch
import re
import sys
import pickle
import wiki_article 
import dictionary_definition
import glob
import modeling
import itertools
import random
import pandas as pd
import numpy as np
import datasets
from torch.nn.utils.rnn import pad_sequence
from dataclasses import dataclass
from io import StringIO
from ipywidgets import interact, interactive, fixed, interact_manual
from transformers import AutoModelWithLMHead, AutoTokenizer
from scipy import stats
import hashlib
from collections import OrderedDict
from types import SimpleNamespace

In [ ]:
def get_checkpoints(base_dir):
    checkpoint_dirs = glob.glob(f"{base_dir}/checkpoint*")
    checkpoint_dirs.sort(key=lambda x: int(x[(x.index("checkpoint-") + len("checkpoint-")):]))
    return checkpoint_dirs
modeling_gpt
def evaluate_lm_checkpoints(base_dir, validation_path):
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    for d in get_checkpoints(base_dir):
        model = AutoModelWithLMHead.from_pretrained(d).to('cuda')
        refined_model_eval = wiki_article.lm_eval(model, tokenizer, validation_path)
        print(f"{d}: {refined_model_eval}")
tokenizer
def evaluate_title_checkpoints(base_dir, validation_path):
    tokenizer = AutoTokenizer.from_pretrained("gpt2")print(parsed_urban_dictionary_scraperpage.body.prettify())
    for d in get_checkpoints(base_dir):
        model = AutoModelWithLMHead.from_pretrained(d).to('cuda')
        refined_model_eval = wiki_article.run_title_evaluation(model, tokenizer, validation_path)
        print(f"{d}: m={refined_model_eval.mean}, v={refined_model_eval.variance}")

# evaluate_lm_checkAutoModelWithLMHead, AutoTokenizer, points("models/wikitext_103_stride_512_v0/", "data/wikitext-103-title-train/wiki_title.valid.raw")
#print(glob.glob("models/wikitext_103_stride_512_v0/*"))

In [3]:
with open(f"data/en_dictionary_parsed_randomized.pickle", "rb") as f:
    parsed_dictionary = pickle.load(f)

In [26]:
potential_blacklist = set()
for word in parsed_dictionary:
    potential_blacklist.add(word.word)
    potential_blacklist.update(word.derivatives)
print(len(parsed_dictionary))
print(len(potential_blacklist))
    
    

98388
114909


In [13]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens(datasets.SpecialTokens.special_tokens_dict())
args = SimpleNamespace()
args.block_size = 768
dataset = datasets.ParsedDictionaryDefinitionDataset(tokenizer, args, None, None, None)

In [16]:
flattened_set = list(itertools.chain.from_iterable(dataset._make_examples(tokenizer, e) for e in parsed_dictionary))

In [120]:
# print(f"{len(flattened_set)} from {len(parsed_dictionary)} entries")
word = tokenizer.encode("vitellogenin")
print(tokenizer.decode(dataset.bos_token_ids  + [1] + dataset.eos_token_ids))
print(tokenizer.decode(tokenizer.encode("<|bod|>\"<|eod|>")))

print(f"\"{tokenizer.decode(dataset.pos_sep_ids)}\"")
tokenizer.decode(dataset._make_examples(tokenizer, parsed_dictionary[0])[0])
# for example in random.choices(flattened_set, k=20):
#     print(tokenizer.decode(example))

<|bod|> " <|eod|>
<|bod|> " <|eod|>
"<|pos|>"


'<|bod|> vitellogenin <|eod|>'

167044 from 98388 entries
<|bod|> Garuda <|bd|> an eaglelike being that Vishnu rides as his mount. <|eod|>
<|bod|> off-screen <|pos|> adverb <|bd|> in real life rather than fictionally in a movie or on television <|be|> happy endings rarely happen off-screen <|eod|>
<|bod|> kt. <|pos|> abbreviation <|bd|> knot(s) <|be|> a cruising speed of 240 kt <|eod|>
<|bod|> notice <|pos|> noun <|bd|> notification or warning of something, especially to allow preparations to be made <|be|> interest rates are subject to fluctuation without notice <|eod|>
<|bod|> rally <|pos|> verb <|bd|> (in tennis and other racket sports) engage in a rally. <|eod|>
<|bod|> deuced <|pos|> adjective [attributive] <|bd|> used for emphasis, especially to express disapproval or frustration <|be|> I'm so deuced fond of you <|be|> I know it's deuced awkward for you <|eod|>
<|bod|> Masuria <|bd|> a low-lying forested region in northeastern Poland. Formerly part of East Prussia, it was assigned to Poland after World War II. 

In [27]:
for example in dataset._make_examples(tokenizer, parsed_dictionary[10430]):
    print(tokenizer.decode(example))

<|bod|> wallpaper <|pos|> noun <|bd|> paper that is pasted in vertical strips over the walls of a room to provide a decorative or textured surface. <|eod|>
<|bod|> wallpaper <|pos|> noun <|bd|> an optional background pattern or picture on a computer or mobile phone screen. <|eod|>
<|bod|> wallpaper <|pos|> verb [with object] <|bd|> apply wallpaper to (a wall or room). <|eod|>


In [ ]:
with open("data/all_words.pickle", "rb") as f:
    #words = pickle.load(f)
    #items = list(words.items())
    random.shuffle(items)
    items = OrderedDict(items)
    
with open("data/all_words_randomized.pickle", "wb") as f:
    pickle.dump(items, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
urban_dictionary_scraper.UrbanDictionaryDataset._make_examples(tokenizer, words[2])

In [ ]:

model = AutoModelWithLMHead.from_pretrained("gpt2").to('cuda')

In [ ]:
unrefined_model_eval = wiki_article.run_title_evaluation(urban_dictionary_scrapermodel, tokenizer, "wikitext-103-raw/wiki.valid.raw")
unrefined_model_eval

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelWithLMHead.from_pretrained("output_103/").to('cuda')

In [ ]:
refined_model_eval = wiki_article.run_title_evaluation(model, tokenizer, "wikitext-103-raw/wiki.valid.raw")
refined_model_eval

In [ ]:
sequence = f"\"TITLE\" is a song collaboration by Chinese artist Pamela Chen and Canadian singer Thomas Dimson, first released independently in March 2020. After gaining popularity amongst the cat community, the single was re-released by major label Columbia Records in May 2020. Pamela describes the song as being originally inspired by her two kittens, Apollo and Bean who once said meow.<bot>"

model =  modeling.GPT2LMHeadWithWeightedLossModel.from_pretrained("models/wikitext-103-raw-title-scale-20-lr5e-5").to("cuda")
input = tokenizer.encode(sequence, return_tensors="pt").to('cuda')
generated = model.generate(input, max_length=100, num_return_sequences=100, temperature=1)

print(f"Prompt text: {sequence}")
for i in range(generated.size()[0]):
    sentence_tokens = generated[i, :].tolist()
    decoded = tokenizer.decode(sentence_tokens)
    m = re.search(r"<bot>(.*?)<eot>", decoded)
    if m:urban_dictionary_scraper
        print(f"{i}) {m.groups(1)}")
    else:
        print(f"{i}) Didn't work")
    

resulting_string = tokenizer.decode(generated.tolist()[0])
# print(resulting_string)

In [ ]:
for entry in entries: 
    m = re.match(r"\s*" + re.escape(entry.title) + r"\d*\s*(\|[^|]*\|)?\s*", entry.entry_str)
    if m:
        trainable_entry = entry.entry_str[m.span()[1]:].strip()
        if not trainable_entry:
            raise RuntimeError(f"Bad entry for {entry.title}: '{entry.entry_str}'")
    else:
        raise RuntimeError(f"Couldn't match {entry.title} on '{entry.entry_str}'")

In [ ]:
dictionary_path = "data/com_apple_MobileAsset_DictionaryServices_dictionaryOSX/69b7ab1cf0f75ad16bf6662b0a77fbfd36b7941f.asset/AssetData/New Oxford American Dictionary.dictionary/Contents/Resources/Body.data"
with open(dictionary_path, "rb") as f:
    valid_words = {e.title.upper() for e in dictionary_definition.DictionaryDefinition.gen_from_apple_dictionary(f)}full_dataset = [
    
]

In [66]:
model =  modeling.GPT2LMHeadWithWeightedLossModel.from_pretrained("models/dictionary-scale-10-lr5e-5").to("cuda")


OSError: Can't load 'models/dictionary-scale-10-lr5e-5'. Make sure that:

- 'models/dictionary-scale-10-lr5e-5' is either a correct model identifier of a community model from 'https://huggingface.co/models' which has a 'config.json' file

- or 'models/dictionary-scale-10-lr5e-5' is a model name in ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl', 'distilgpt2']

- or 'models/dictionary-scale-10-lr5e-5' is the correct path to a directory containing a 'config.json' file

In [ ]:
words = dictionary_definition.generate_words(
    tokenizer, model, allow_proper_nouns=False, blacklist=valid_words, num=1000, max_iterations=40
)
words.sort(key=lambda x: x.title)
for w in words:
    print(f"{w} {w.entry_str}")

In [ ]:
with open("words.tsv", "w") as f:
    for word in words:
        f.write(f"{word.title}\t{word.entry_str}\n")

In [134]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens(datasets.SpecialTokens.special_tokens_dict())
blacklist = set((x.lower() for x in itertools.chain.from_iterable(
    [e.word] + e.derivatives
    for e in pickle.load(open(f"data/en_dictionary_parsed_randomized.pickle", "rb")))
))
model = AutoModelWithLMHead.from_pretrained("models/en_dictionary_parsed_lr_00005/checkpoint-50000").to("cuda")

In [139]:
words = datasets.ParsedDictionaryDefinitionDataset.generate_words(
    tokenizer, model, max_iterations=10, blacklist=blacklist, do_example_expansion=True,
)
for word in words:
    print(f"{word.word} /{word.pos}/")
    print(f"\t{word.definition}")
    print(f"\t\"{word.example}\"{' [e]' if word.from_example_expansion else ''}")
        
    print()

Nilox /noun/
	a member of a North American Indian people living in the Carboniferous–Piloches Mountains in western New Mexico.
	"when the Nilox was sent up by the Indians, the upper part of their bodies were burned to make it look like sheep meat" [e]

crestle /noun/
	an ornamental design resembling a cup.
	"the cupboard in front was a throne on a crestle" [e]

pancre /noun/
	a hollowed wooden structure resembling pancake in shape.
	"the pancre of his stomach" [e]

m.c. /abbreviation/
	mordant.
	"we're looking at two new titles from the m.c. of Shakespeare" [e]

harpaloo /noun/
	a Hawaiian shrub of the berry family which has aromatic flowers and is widely grown for fodder.
	"many of the country's most popular harpaloo shrubs are planted with mulch" [e]

socktail /noun/
	a piece of soft, sweet hair cut from small sashes, the hair of a female fish, or a person's breast.
	"the men were given a free sample of the popular Socktail at the aquarium" [e]

gir /noun/
	a small edible fruit of th

from datasets import SpecialTokens
"""
input_str = f"{tokenizer.bos_token}"
input_str = "<|bod|>corner<|pos|>noun<|bd|>a point or space in a hierarchy that is within the order to which it moves along the axis.<|eod|>"
input = tokenizer.encode(input_str, return_tensors="pt").to("cuda")
max_length = 512

generated = model.generate(
    input_ids=input, 
    max_length=max_length, 
    num_return_sequences=5, 
    temperature=1.0,
    top_k=1000,
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_ids=tokenizer.eos_token_id,
    do_sample=True,
)

break_specials = [
    SpecialTokens.BOS_TOKEN, SpecialTokens.EOS_TOKEN, SpecialTokens.DEFINITION_SEP,
    SpecialTokens.EXAMPLE_SEP, SpecialTokens.TOPIC_SEP, SpecialTokens.POS_SEP 
]
break_special_ids = [tokenizer.encode(e, add_prefix_space=False)[0] for e in break_specials]
break_special_token_map = {s: i for s, i in zip(break_specials, break_special_ids)}


for i in range(generated.size()[0]):
    sentence_tokens = generated[i, :].tolist()
    
    
    accum = []
    last_special = None
    sep_map = {}
    for token_id in sentence_tokens:
        if token_id in break_special_ids:
            if last_special is not None:
                sep_map[last_special] = accum
                accum = []
                last_special = token_id
            else:
                last_special = token_id
        else:
            accum.append(token_id)
            
    sep_map[last_special] = accum
    accum = []
    
    decode_sep_map = {
        tokenizer.decode([k]): tokenizer.decode(v) for k, v in sep_map.items()
    }
        
    print(decode_sep_map)
    
    # decoded = tokenizer.decode([e for e in sentence_tokens if e != tokenizer.pad_token_id])
    print(decoded)  
"""


In [95]:
tokenizer.decode(tokenizer.encode("a bc", add_prefix_space=False))

'a bc'

In [71]:
tokenizer.special_tokens_map

{'bos_token': '<|bod|>',
 'eos_token': '<|eod|>',
 'unk_token': '<|endoftext|>',
 'pad_token': '<|pad|>',
 'additional_special_tokens': ['<|bd|>', '<|be|>', '<|pos|>', '<|bto|>']}

In [55]:
blacklist = set(e.title for e in pickle.load(open("data/all_words.pickle", "rb")).values())

In [66]:
model =  modeling.GPT2LMHeadWithWeightedLossModel.from_pretrained(
    "models/urban_dictionary_cleaned_top_def_mu02_lr_0_000005_tw40"
).to("cuda")
tw40_words = urban_dictionary_scraper.generate_words(
    tokenizer,
    model,
    blacklist=blacklist,
    num=100,
)

Generation: tried 350, failed 102 (0.29), no word in example 121 (0.49), filtered from blacklist 12 (0.09)


In [81]:
pickle.dump(tw1_words, open("data/labeling/tw1_words.pickle", "wb"), protocol=pickle.HIGHEST_PROTOCOL)
pickle.dump(tw40_words, open("data/labeling/tw40_words.pickle", "wb"), protocol=pickle.HIGHEST_PROTOCOL)

In [72]:
df = pd.DataFrame(
    [
        (
            word.word,
            word.definition,
            word.example.replace(,
            "tw1" if i < len(tw1_words) else "tw2",
        )
        for i, word in enumerate(itertools.chain(
            tw1_words,
            tw40_words
        ))
    ],
    columns=("word", "definition", "example", "dataset")
)

In [74]:
sample = df.sample(frac=1)

In [78]:
sample_no_dataset = sample[:]
sample_no_dataset.to_csv("fun.csv", index=False, columns=["word", "definition", "example"])

In [80]:
interact()

In [32]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens(datasets.SpecialTokens.special_tokens_dict())
model =  AutoModelWithLMHead.from_pretrained("models/en_dictionary_parsed_lr_00005/checkpoint-50000").to("cuda")